In [1]:
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [88]:
filenames = ['sixtysix/sixtysix.A','sixtysix/sixtysix.B','sixtysix/sixtysix.C']

for aname in filenames:
    outstring = []
    with open(aname,'rb') as f:
    #hexdata = f.read().hex()
        tHex = f.read(1).hex()
        index = 0
        aline = []
        while tHex:

            if index%2==0 and index!=0:
                index = 0
                outstring.append(aline)
                aline=[]
                aline.append(int(tHex,base=16))
                index +=1
            else:
                aline.append(int(tHex,base=16))
                index +=1

            tHex = f.read(1).hex()
    df_outstring = pd.DataFrame(outstring)
    df_outstring.to_csv(aname+'_V2.csv',index=False)

In [4]:
# Time from sixtysix.A

df_a = pd.read_csv('sixtysix/sixtysix.A_V2.csv')
df_a = df_a[1:].copy()
print(df_a.shape)
df_a_5 = []
aline = []
index = 0
for k, v in df_a[1:].iterrows():
    if index!=0 and index%5==0:
        df_a_5.append(aline)
        aline = []
        index = 0
    aline.extend(v)
    index +=1
    
if len(aline)>0:
    df_a_5.append(aline)

df_a_5 = pd.DataFrame(df_a_5)
df_a_5 = df_a_5.fillna(value=0)
df_a_5['time'] = df_a_5[[0,1,2,3]].apply(lambda s: sum([s[i]*256**(3-i) for i in range(4)]),axis=1)

def AdjustTime(s):
    res = s%6
    if res >3 or ( res==3 and (s//6)%2==1):
        s = s +(6-res)
    elif res >0:
        s = s - res
    
    return s
        
        
        

df_a_5['adjust'] = df_a_5['time'].apply(lambda s: AdjustTime(s))
df_a_5['adjust_time'] = df_a_5['adjust'].apply(lambda s: s//60000)
df_a_5.to_csv('sixtysix/sixtysix.A_V10.csv',index=False)      

(27158, 2)


In [91]:
# Columns from sixtysix.B

df_b = pd.read_csv('sixtysix/sixtysix.B_V2.csv')
df_b_3 = []
aline = []
index = 0
for k, v in df_b.iterrows():
    if index!=0 and index%3==0:
        df_b_3.append(aline)
        aline = []
        index = 0
    aline.extend(v)
    index +=1
    
if len(aline)>0:
    df_b_3.append(aline)
    
df_b_3 = pd.DataFrame(df_b_3)
df_b_3 = df_b_3.fillna(value=0)
df_b_3['columns'] = df_b_3[[0,1]].apply(lambda s: s[0]+s[1]*256,axis=1)
df_b_3['values'] = df_b_3[[2,3,4,5]].apply(lambda s: s[2]+s[3]*256+s[4]*256*256+s[5]*256*256*256,axis=1)
df_b_3.to_csv('sixtysix/sixtysix.B_V6.csv', index=False)

df_b_3.head()

,0,1,2,3,4,5,columns,values
0,143,3,244,12,0.0,0.0,911,3316.0
1,143,3,54,13,0.0,0.0,911,3382.0
2,143,3,121,13,0.0,0.0,911,3449.0
3,143,3,190,13,0.0,0.0,911,3518.0
4,143,3,224,13,0.0,0.0,911,3552.0


In [92]:
df_b_3.tail()

,0,1,2,3,4,5,columns,values
58701,215,2,1,0,0.0,0.0,727,1.0
58702,215,2,1,0,0.0,0.0,727,1.0
58703,215,2,1,0,0.0,0.0,727,1.0
58704,215,2,1,0,0.0,0.0,727,1.0
58705,215,2,1,0,0.0,0.0,727,1.0


In [5]:
start_pos = [0,#1, 303
 207,#2 #311
 2678,#21 #316
 3059,#20 #371
 4408,#13 #375
 4187,#16 #388
 2771,
 2197,
 3373,
 2367,
 3283,
 2302,
 1446,
 1565,
 1643,
 2356,
 2447,
 3058,
 643,
 2566,
 2161,
 1393,
 1809,
 1383,
 2582,
 2566,
 3067,
 2339,
 3997,
 2387,
 3815,
 285,
 407,
 1165,
 1996,
 751,
 4113,
 3964,
 2513,
 3632,
 0,
 411,
 2609,
 1662,
 2053,
 3461,
 2463,
 380,
 934]

In [6]:
import numpy as np
df_b_3 = pd.read_csv('sixtysix/sixtysix.B_V6.csv')

df_grouped = df_b_3.groupby('columns')
column_dict = {}
index = 1
for k, g in df_grouped:
    #print(k, g)
    try:
        column_dict[k] = np.hstack((np.zeros(start_pos[index]),np.array(g['values'].values),np.zeros(5432-start_pos[index]-g['values'].shape[0])))  
        index += 1
    except:
        print(index)

In [12]:
df_b_result = pd.DataFrame.from_dict(column_dict)
df_b_result['time'] = df_a_5['adjust'].tolist()
df_b_result['time'] = df_b_result['time'].apply(lambda s: s*1.0/60000)
df_b_result = df_b_result[['time']+list(df_b_result.columns[:-1])]
df_b_result.head()

,time,303,311,316,371,375,388,402,409,413,...,910,911,915,916,924,933,943,948,960,969
0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3382.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3518.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3552.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_b_result.to_csv('data/mass_final.csv',index=False)